In [1]:
!pip install kagglehub

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

100%|██████████| 8.89M/8.89M [00:03<00:00, 3.08MB/s]

Extracting files...


Path to dataset files: C:\Users\DELL\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\DELL\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2


In [12]:
import numpy as np
import pandas as pd

movies = pd.read_csv(r'C:\Users\DELL\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_movies.csv')
credits = pd.read_csv(r'C:\Users\DELL\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_credits.csv')
movies = movies.merge(credits,on='title')

movies = movies[['movie_id','title','overview','genres','keywords','cast','crew']]

movies['keywords'] = movies['keywords'].apply(convert)

import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 


movies['cast'] = movies['cast'].apply(convert)
movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 


movies['crew'] = movies['crew'].apply(fetch_director)
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new = movies.drop(columns=['overview','genres','keywords','cast','crew'])
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
    
vector = cv.fit_transform(new['tags']).toarray()
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
new[new['title'] == 'The Lego Movie'].index[0]
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)
import pickle
pickle.dump(new,open('movie_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))
    

NameError: name 'convert' is not defined

In [13]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Load datasets
movies = pd.read_csv(r'C:\Users\DELL\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_movies.csv')
credits = pd.read_csv(r'C:\Users\DELL\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_credits.csv')

# Merge on title
movies = movies.merge(credits, on='title')

# Keep only useful columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

# ---------- Step 1: Define required conversion functions ----------

# Converts stringified list of dicts to list of names
def convert(text):
    L = []
    try:
        for i in ast.literal_eval(text):
            L.append(i['name'])
    except:
        pass
    return L

# Converts cast string to top 3 cast names
def convert3(text):
    L = []
    counter = 0
    for i in ast.literal_eval(text):
        if counter < 3:
            L.append(i['name'])
            counter += 1
    return L

# Fetch director name from crew
def fetch_director(text):
    L = []
    try:
        for i in ast.literal_eval(text):
            if i['job'] == 'Director':
                L.append(i['name'])
                break
    except:
        pass
    return L

# Removes spaces from multi-word names
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ", ""))
    return L1

# ---------- Step 2: Apply functions to columns ----------

movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert3)
movies['crew'] = movies['crew'].apply(fetch_director)

# Preprocess all text columns
movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

# Split overview into words
movies['overview'] = movies['overview'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Combine all text into a single "tags" column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

# Drop unused columns
new = movies[['movie_id', 'title', 'tags']]
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

# ---------- Step 3: Vectorization using Bag of Words ----------
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()

# ---------- Step 4: Similarity Calculation ----------
similarity = cosine_similarity(vector)

# ---------- Step 5: Recommendation Function ----------
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    print(f"\nTop 5 recommendations for '{movie}':")
    for i in distances[1:6]:
        print(new.iloc[i[0]].title)

# ---------- Step 6: Save the model ----------
pickle.dump(new, open('movie_list.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))


C:\Users\DELL\AppData\Local\Temp\ipykernel_14224\3748212641.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['tags'] = new['tags'].apply(lambda x: " ".join(x))


In [1]:
import numpy as np
import pandas as pd
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import pickle

# Load datasets
movies = pd.read_csv(r'C:\Users\DELL\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_movies.csv')
credits = pd.read_csv(r'C:\Users\DELL\.cache\kagglehub\datasets\tmdb\tmdb-movie-metadata\versions\2\tmdb_5000_credits.csv')

# Merge on title
movies = movies.merge(credits, on='title')

# Select required columns
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew', 'vote_average', 'popularity']]

# Helper functions
def convert(text):
    L = []
    try:
        for i in ast.literal_eval(text):
            L.append(i['name'])
    except:
        pass
    return L

def convert3(text):
    L = []
    counter = 0
    try:
        for i in ast.literal_eval(text):
            if counter < 3:
                L.append(i['name'])
                counter += 1
    except:
        pass
    return L

def fetch_director(text):
    L = []
    try:
        for i in ast.literal_eval(text):
            if i['job'] == 'Director':
                L.append(i['name'])
                break
    except:
        pass
    return L

def collapse(L):
    return [i.replace(" ", "") for i in L]

# Apply transformations
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert3)
movies['crew'] = movies['crew'].apply(fetch_director)

movies['cast'] = movies['cast'].apply(collapse)
movies['crew'] = movies['crew'].apply(collapse)
movies['genres'] = movies['genres'].apply(collapse)
movies['keywords'] = movies['keywords'].apply(collapse)

movies['overview'] = movies['overview'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Create tags column
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new = movies[['movie_id', 'title', 'tags', 'vote_average', 'popularity']]
new['tags'] = new['tags'].apply(lambda x: " ".join(x))

# Vectorize tags
cv = CountVectorizer(max_features=5000, stop_words='english')
vector = cv.fit_transform(new['tags']).toarray()

# Cosine similarity
similarity = cosine_similarity(vector)

# KMeans clustering (ML model)
kmeans = KMeans(n_clusters=20, random_state=42)
kmeans.fit(vector)
new['cluster'] = kmeans.labels_

# Enhanced recommendation with popularity and rating sorting
def recommend(movie):
    index = new[new['title'] == movie].index[0]
    distances = list(enumerate(similarity[index]))
    distances = sorted(distances, reverse=True, key=lambda x: x[1])
    
    movie_list = []
    for i in distances[1:11]:  # Top 10 similar movies
        movie_list.append((
            new.iloc[i[0]].title,
            movies.iloc[i[0]]['vote_average'],
            movies.iloc[i[0]]['popularity']
        ))
    
    # Sort by rating then popularity
    movie_list = sorted(movie_list, reverse=True, key=lambda x: (x[1], x[2]))
    
    print(f"\nTop 5 recommended movies for '{movie}' (sorted by rating/popularity):")
    for movie, rating, pop in movie_list[:5]:
        print(f"{movie} — Rating: {rating}, Popularity: {pop:.2f}")

# Cluster-based alternative recommendation
def recommend_by_cluster(movie):
    index = new[new['title'] == movie].index[0]
    cluster_label = new.iloc[index]['cluster']
    cluster_movies = new[(new['cluster'] == cluster_label) & (new.index != index)]
    
    print(f"\nCluster-based recommendations for '{movie}':")
    for title in cluster_movies['title'].sample(min(5, len(cluster_movies))).values:
        print(title)

# Save model and data
import os

# 🔒 Create model folder if it doesn't exist
if not os.path.exists('model'):
    os.makedirs('model')

# 💾 Save processed data and similarity matrix
pickle.dump(new, open('model/movie_list.pkl', 'wb'))
pickle.dump(similarity, open('model/similarity.pkl', 'wb'))

print("✅ Saved model/movie_list.pkl and similarity.pkl")

# Example usage
recommend("The Lego Movie")
recommend_by_cluster("The Lego Movie")


C:\Users\DELL\AppData\Local\Temp\ipykernel_864\1622920493.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['tags'] = new['tags'].apply(lambda x: " ".join(x))
C:\Users\DELL\AppData\Local\Temp\ipykernel_864\1622920493.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['cluster'] = kmeans.labels_


✅ Saved model/movie_list.pkl and similarity.pkl

Top 5 recommended movies for 'The Lego Movie' (sorted by rating/popularity):
The Book of Life — Rating: 7.3, Popularity: 34.89
National Lampoon's Vacation — Rating: 7.1, Popularity: 22.84
Despicable Me 2 — Rating: 7.0, Popularity: 136.89
The Croods — Rating: 6.8, Popularity: 64.18
The Boxtrolls — Rating: 6.6, Popularity: 30.66

Cluster-based recommendations for 'The Lego Movie':
Agent Cody Banks 2: Destination London
Tombstone
The Hunger Games: Mockingjay - Part 2
City of Ember
300: Rise of an Empire
